In [2]:
from medusa.model.medusa_model import MedusaConfig


config = MedusaConfig.from_pretrained('./train/0625_medusa_mlp_Meta-Llama-3-8B-Instruct_medusa_5_lr_0.0001_layers_1')
config

MedusaConfig {
  "base_model_name_or_path": "/home/camus/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6",
  "medusa_num_heads": 5,
  "medusa_num_layers": 1,
  "transformers_version": "4.41.2",
  "version": "2"
}

In [4]:
from fastchat.model.model_adapter import get_conversation_template


conv = get_conversation_template(config.base_model_name_or_path)
conv

Conversation(name='llama-3', system_template='<|start_header_id|>system<|end_header_id|>\n\n{system_message}<|eot_id|>', system_message='', roles=('user', 'assistant'), messages=[], offset=0, sep_style=<SeparatorStyle.LLAMA3: 8>, sep='', sep2=None, stop_str='<|eot_id|>', stop_token_ids=[128001, 128009], max_image_size_mb=None)

In [5]:
import json


data = json.load(open('./assets/alpaca_eval.json'))
len(data)

805

In [6]:
sample = data[0]
sample

{'dataset': 'helpful_base',
 'instruction': 'What are the names of some famous actors that started their careers on Broadway?',
 'output': 'Some famous actors that started their careers on Broadway include: \n1. Hugh Jackman \n2. Meryl Streep \n3. Denzel Washington \n4. Julia Roberts \n5. Christopher Walken \n6. Anthony Rapp \n7. Audra McDonald \n8. Nathan Lane \n9. Sarah Jessica Parker \n10. Lin-Manuel Miranda',
 'generator': 'text_davinci_003'}

In [7]:
conv.messages = []
conv.append_message(conv.roles[0], sample["instruction"])
conv.append_message(conv.roles[1], "")

prompt = conv.get_prompt()
prompt

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat are the names of some famous actors that started their careers on Broadway?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [8]:
import torch
from medusa.model.medusa_model import MedusaModel


model = MedusaModel.from_pretrained(
    './train/0625_medusa_mlp_Meta-Llama-3-8B-Instruct_medusa_5_lr_0.0001_layers_1',
    medusa_num_heads=5,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
)
tokenizer = model.get_tokenizer()

model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.45s/it]
Some weights of MedusaModelLlama were not initialized from the model checkpoint at /home/camus/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6 and are newly initialized: ['medusa_head.0.0.linear.bias', 'medusa_head.0.0.linear.weight', 'medusa_head.0.1.weight', 'medusa_head.1.0.linear.bias', 'medusa_head.1.0.linear.weight', 'medusa_head.1.1.weight', 'medusa_head.2.0.linear.bias', 'medusa_head.2.0.linear.weight', 'medusa_head.2.1.weight', 'medusa_head.3.0.linear.bias', 'medusa_head.3.0.linear.weight', 'medusa_head.3.1.weight', 'medusa_head.4.0.linear.bias', 'medusa_head.4.0.linear.weight', 'medusa_head.4.1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MedusaModelLlama(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Li

In [9]:
input_ids = tokenizer([prompt]).input_ids
input_ids = torch.as_tensor(input_ids).cuda()
input_ids

tensor([[128000, 128000, 128006,    882, 128007,    271,   3923,    527,    279,
           5144,    315,   1063,  11495,  20142,    430,   3940,    872,  31133,
            389,  37776,     30, 128009, 128006,  78191, 128007,    271]],
       device='cuda:0')

In [11]:
from medusa.model.kv_cache import initialize_past_key_values
from medusa.model.utils import reset_medusa_mode


past_key_values, past_key_values_data, current_length_data = initialize_past_key_values(model.base_model)
model.past_key_values = past_key_values
model.past_key_values_data = past_key_values_data
model.current_length_data = current_length_data

model.current_length_data.zero_()
reset_medusa_mode(model)

In [12]:
with torch.inference_mode():
	medusa_logits, outputs, logits = model(
		input_ids, past_key_values=past_key_values, output_orig=True, medusa_forward=True
	)

medusa_logits.shape, outputs, logits.shape

(torch.Size([5, 1, 26, 128256]),
 BaseModelOutputWithPast(last_hidden_state=tensor([[[ 4.0078, -0.5010, -1.9932,  ..., -3.7461,  0.8423,  2.6992],
          [ 4.0078, -0.5010, -1.9932,  ..., -3.7461,  0.8423,  2.6992],
          [ 2.1016, -0.5078, -1.7432,  ...,  0.0694,  1.4648, -0.9570],
          ...,
          [ 0.7495,  0.1599,  3.6270,  ...,  2.4043, -0.1870,  0.8550],
          [-1.3467, -1.1621,  1.2295,  ...,  0.3220, -1.3652,  1.5195],
          [-0.6021,  1.4229,  4.0508,  ...,  0.8022,  0.1729,  0.6201]]],
        device='cuda:0', dtype=torch.float16), past_key_values=(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None), hidden_states=None, attentions=None),
 torch.Size([1, 26, 128256]))

In [15]:
_, medusa_topk = medusa_logits[...,-1,:].topk(20, dim=-1)
_.shape, medusa_topk.shape

(torch.Size([5, 1, 20]),
 torch.Size([5, 1, 20]),
 tensor([[[  6553,  30783,  97049,  81813,  12152,  60812,  35948,  28033,
             6455,  93277,  85174,  68799,  62353,  75046, 126489, 115320,
            85165,  49820, 109946,  29285]],
 
         [[ 61236,  37390,  33318,  59865, 124901,  78783,  30653,   6545,
           114540,  87370,   1160, 126758, 125714,  68845,    294,  64895,
            27206,  39817,   4411, 119565]],
 
         [[ 25063,  99170,  40558,  77186,  80921,    285,  78408, 117483,
            79795,  56068,  47971,  55309,  19549,  87357,  43742,  51086,
            14950,  69071,  66109,  80492]],
 
         [[ 95892,  81706, 111001,  92549,  12849,  60000, 100809,   3569,
            11085,  68411,  44347,  84774,  99830,  76173,  56963,  58317,
           101262,  41727,  95236, 110330]],
 
         [[ 89963,  37407,  22819, 100229, 117782, 119140,  40855,   8911,
            36413, 113467, 112399, 126557,  12415,  17055, 114597,  65348,
            

In [17]:
[', '.join([tokenizer._tokenizer.id_to_token(id) for id in medusa_topk[i, 0]]) for i in range(5)]

['iforn, Payload, æŃ³, Ġipad, ĠCanadian, (delete, .Dep, ĠLegend, si, ĠHLS, ayah, ISK, Ġcasi, Hyper, ĠÚ¯Ø°Ø§Ø±ÛĮ, é£², ĠNEG, Ġscreenshots, ĠNhá»¯ng, aveled',
 'ĠCater, Ġrhythm, Ġdevastating, .TAG, Ġhá»©, lsa, -Ch, .web, ĠÐ²Ð¾Ð·Ð´ÐµÐ¹ÑģÑĤÐ², Ġconfiscated, Ġlist, ãĤĤãģªãģĦ, Ġê³łëł¤, ĠCunning, Ġd, Ġhailed, ĠHorn, -defined, Ġmor, ÑĢÐ¸Ð¸',
 '-set, ĠBinaryTree, Ġbees, ĉReturns, Ġkitty, is, Ġesi, :::|, Ace, Slug, ¾ç¤º, Ġbargaining, shall, ĠQSize, Removed, connections, CLU, Grant, .Temp, Ġterrestrial',
 '):?>Ċ, svp, Æ°á»Ŀn, _blocking, Ġcompute, _returns, Ġquan, ++)Ċ, .java, avl, ĠisActive, ĠcreateContext, igmatic, ByExample, [state, ĠÐ¶, Ð°Ð½Ñĸ, =l, ()?;Ċ, ĠÑĢÐµÐ°Ðº',
 '>{", ĠFabric, OTAL, idders, ĠÙĨØ³Ø¨Ø©, adla, ĠDepart, Ġappend, Ġprima, ĠAnadolu, ĠãĦ, ãĢįãĢĤ, Ð¹, Ġhydro, ìĹĺ, MethodInfo, Ġforfeiture, ĠÑĨÑĸÐ», à¸µà¸¢, Ġà¤Ńà¤Ĺ']

In [19]:
input_id = logits[:, -1:].argmax(dim=-1)
input_id, tokenizer._tokenizer.id_to_token(input_id.item())

(tensor([[2]], device='cuda:0'), '#')